In [ ]:
using CUDA
using BenchmarkTools
using QuantumClifford

In [ ]:
#to determine what devices we are running on and attempt to determine comparability of hardware.
CUDA.device()

In [ ]:
Sys.cpu_info()

In [ ]:
#Original GPU apply function, for qubits<=64
function apply_gate_gpu!(s::CuArray{UInt64, 2}, c::CuArray{UInt64, 2})
    # Kernel function
    function kernel(s, c, num_bits)
        i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
        if i <= num_bits
            tempX = UInt64(0)
            tempZ = UInt64(0)
            tempSX = s[1, i]
            tempSZ = s[2, i]
            #@cuprintln("tempSX %d\n", tempSX)
            #@cuprintln("tempSZ %d\n", tempSZ)
            for j in 1:num_bits
                if tempSX & 1 == 1
                    #=
                    @cuprintf("Here X %d\n", i)
                    @cuprintln(j)
                    @cuprintln(c[1,j])
                    @cuprintf("After here X%d\n", i)
                    =#
                    tempX ⊻= c[1, j]
                    #hard coded for 4 qubits for now
                    tempZ ⊻= c[2, j]
                    #@cuprintln("tempX %d\n", tempX)
                    #@cuprintln("tempZ %d\n", tempZ)
                end
                if tempSZ & 1 == 1
                    #=
                    @cuprintf("Here Z %d\n", i)
                    @cuprintln(j)
                    @cuprintln(c[2,j])
                    @cuprintf("After here Z%d\n", i)
                    =#
                    tempX ⊻= c[1, j+num_bits]
                    #hard coded for 4 qubits for now
                    tempZ ⊻= c[2, j+num_bits]
                    #@cuprintln("tempX %d\n", tempX)
                    #@cuprintln("tempZ %d\n", tempZ)
                end
                tempSX >>= 1
                tempSZ >>= 1
            end
            s[1, i] = tempX
            s[2, i] = tempZ
        end
        return
    end

    # Check if input exceeds 64 bits
    if size(s, 2) > 64
        println("MORE THAN 64")
        return
    end

    # Launch the kernel
    num_bits = size(s, 2)
    #println(num_bits)
    threads_per_block = 64
    blocks = ceil(Int, num_bits / threads_per_block)
    @cuda threads=threads_per_block blocks=blocks kernel(s, c, num_bits)
end


In [ ]:
#Testing apply_gate_gpu!
num_qbits = 64
#can change num_qbits to any number between 1 and 64 inclusive
cliff = random_clifford(num_qbits)
#The below comments are useful for visualizing how the data is actually stored
#println(cliff.tab.xzs)
#println(map(bitstring, cliff.tab.xzs))
#println(map(bitstring, cliff.tab.xzs[1, :]))
#println(map(bitstring, cliff.tab.xzs[1,3]))
#dump(cliff) VERY useful for finding functions
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
s = random_stabilizer(num_qbits)
s_gpu = CuArray(s.tab.xzs)
#println(s.tab.xzs)

correct = apply!(s, cliff, phases=false)
correct = correct.tab.xzs
println("Correct answer: $correct")
apply_gate_gpu!(s_gpu, c_gpu)
println(s_gpu)
gpu_result = Array(s_gpu)
# Verify correctness
@assert correct == gpu_result "GPU and CPU results do not match!"
println("Verification successful!")

In [ ]:
#Benchmarking apply_gate_gpu!
num_qbits = 64
#can change num_qbits to anything from 1-64
cliff = random_clifford(num_qbits)
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
s = random_stabilizer(num_qbits)
s_gpu = CuArray(s.tab.xzs)
tempS = tab(s).xzs
tempCliff = cliff.tab.xzs
println("Benchmarking GPU")
#DON'T FORGET CUDA.@sync, otherwise will just benchmark the time it takes to transfer over data
gpu_time = @benchmark CUDA.@sync apply_gate_gpu!(s_gpu, c_gpu)
println("GPU")
println(gpu_time)
show(stdout, "text/plain", gpu_time)
# CPU Benchmark
println()
println("Benchmarking CPU")
cpu_time = @benchmark apply!(s, cliff, phases=false)
println("CPU")
println(cpu_time)
show(stdout, "text/plain", cpu_time)
println()

In [ ]:
#GPU again for 65+ qubits (really 1-infinite qubits)

#Profiling tools (NVIDIA) could make speed-ups, help determine what the bottleneck is. More for educational value and something I didn't have time to look into
function apply_gate_gpu_64plus!(s, c, num_bits)
    #z_start is where the z's start for example if there are 67 bits it will be 3, if there are 129 bits it will be 4 etc.
    #x_stop is just 1 before z_start
    function kernel(s, c, num_bits, numBitstrings, tempXZArr, x_stop, z_start)
        i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
        if i <= num_bits
            for j in 1:64
                #x's loop
                for k in 1:x_stop
                    if s[k, i] & 1 == 1
                        for l in 1:numBitstrings
                            #before it was just c[l, j], but now a bit more complicated indexing
                            tempXZArr[i, l] ⊻= c[l, j+(k-1)*64]
                        end
                    end
                    s[k, i] >>= 1
                end
                #z's loop
                for k in z_start:numBitstrings
                    if s[k, i] & 1 == 1
                        for l in 1:numBitstrings
                             #before it was just c[l, j+ num_bits], but now a bit more complicated
                            tempXZArr[i, l] ⊻= c[l, j+num_bits+(k-z_start)*64]
                        end
                    end
                    s[k, i] >>= 1
                end
            end
            for a in 1:numBitstrings
                s[a, i] = tempXZArr[i, a]
            end
        end
        return 
    end


    # Launch the kernel
    #currently we are passed in number of bits, which I don't think is unreasonable, although could be done away with, with some calculations)
    numBitstrings = ceil(Int, num_bits/64)*2
    x_stop = Int(numBitstrings / 2)
    z_start = Int((numBitstrings / 2) + 1)
    tempXZArr = CUDA.fill(UInt64(0), num_bits, numBitstrings)
    #starting XZArr at 0's
    threads_per_block = 64
    blocks = ceil(Int, num_bits / threads_per_block)
    @cuda threads=threads_per_block blocks=blocks kernel(s, c, num_bits, numBitstrings, tempXZArr, x_stop, z_start)
end


In [ ]:
#Testing apply_gate_gpu_64plus!
num_qbits = 20000
#tensor_pow option used if we want to test a larger state/gate since random_clifford takes a long time as input gets large >2000
#Otherwise can use the below two lines
#cliff = random_clifford(num_qbits)
#s = random_stabilizer(num_qbits)
#If using tensor_pow need to adjust values accordingly, i.e. for 10,000 qubits do...
#num_qbits = 10000, 
#cliff = tensor_pow(random_clifford(1000),10) or tensor_pow(random_clifford(500),20) 
#s = s = tensor_pow(random_stabilizer(1000),10)
cliff = tensor_pow(random_clifford(1000),20)
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
s = tensor_pow(random_stabilizer(1000),20)
s_gpu = CuArray(s.tab.xzs)


correct = apply!(s, cliff, phases=false)
correct = correct.tab.xzs
#println("Correct answer: $correct")
apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_result = Array(s_gpu)
# Verify correctness
@assert correct == gpu_result "GPU and CPU results do not match!"
println("Verification successful!")

In [ ]:
#Benchmarking apply_gate_gpu_64plus!
num_qbits = 1024
cliff = random_clifford(num_qbits) #have same tensor_pow option as previous cell
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
s = random_stabilizer(num_qbits)
s_gpu = CuArray(s.tab.xzs)
tempS = tab(s).xzs
tempCliff = cliff.tab.xzs
correct = apply!(s, cliff, phases=false)
correct = correct.tab.xzs
# GPU Benchmark
println("Benchmarking GPU")
#DON'T FORGET CUDA.@sync 
gpu_time = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
println("GPU")
println(gpu_time)
show(stdout, "text/plain", gpu_time)
# CPU Benchmark
println("Benchmarking CPU")
cpu_time = @benchmark apply!(s, cliff, phases=false)
println("CPU")
println(cpu_time)
show(stdout, "text/plain", cpu_time)
println()

In [ ]:
#trying to get an idea of what exactly is occupying the most time, not very successful but keeping here in case others can parse it better than I
using Profile
num_qbits = 20000
#cliff = random_clifford(num_qbits)
cliff = tensor_pow(random_clifford(1000),20)
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
#s = random_stabilizer(num_qbits)
s = tensor_pow(random_stabilizer(1000),20)
s_gpu = CuArray(s.tab.xzs)
@profile apply!(s, cliff, phases=false)
Profile.print()

In [ ]:
num_qbits = 20000
#cliff = random_clifford(num_qbits)
cliff = tensor_pow(random_clifford(1000),20)
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
#s = random_stabilizer(num_qbits)
s = tensor_pow(random_stabilizer(1000),20)
s_gpu = CuArray(s.tab.xzs)
tempS = tab(s).xzs
tempCliff = cliff.tab.xzs
correct = apply!(s, cliff, phases=false)
correct = correct.tab.xzs
#println("Correct answer: $correct")
# GPU Benchmark
println("Benchmarking GPU")
#DON'T FORGET CUDA.@sync 
gpu_time = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
println("GPU")
println(gpu_time)
show(stdout, "text/plain", gpu_time)
# CPU Benchmark
println("Benchmarking CPU")
cpu_time = @benchmark apply!(s, cliff, phases=false)
println("CPU")
println(cpu_time)
show(stdout, "text/plain", cpu_time)
println()

In [ ]:
# Ensure CPU is single-threaded
#it should be noted we select 2 threads here because we are running on the Unity cluster. 
#It is advised, prior to startup, that at least 2 CPU threads are used so one can focus on simply running the webapplication. 
#Thus we select 2 threads, also when I tried to run with only 1 it would fail, so that extra thread seems to be focused so
#This for loop is giving issues, it worked once but then threw errors from then on, not entirely sure why so just leaving it here as it is nicer than listing everything out if the issue can be resolved
#Still not entirely sure why it doesn't work sometimes, but as of now if I define c_gpu and s_gpu somewhere previously and run that codeblock this seems to run but keep that s_gpu and c_gpu for the whole time
#=
using LinearAlgebra
#BLAS.set_num_threads(2)

# Define problem sizes
problem_sizes = [64, 512, 1024, 2000, 5000, 10000, 20000]
cpu_times = Float64[]
gpu_times = Float64[]

for num_qbits in problem_sizes
    println("\nBenchmarking with num_qbits = $num_qbits")
    if num_qbits >= 2000
        cliff = tensor_pow(random_clifford(1000),num_qbits/1000)
        s = tensor_pow(random_stabilizer(1000),num_qbits/1000)
    else
        cliff = random_clifford(num_qbits)
        s = random_stabilizer(num_qbits)
    end
    c_gpu = CuArray(cliff.tab.xzs)  
    s_gpu = CuArray(s.tab.xzs)
    

    # CPU Benchmark
    cpu_bench = @benchmark apply!(s, cliff, phases=false)
    cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
    println("CPU time: $cpu_time seconds")
    push!(cpu_times, cpu_time)

    # GPU Benchmark
    gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
    gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
    println("GPU time: $gpu_time seconds")
    push!(gpu_times, gpu_time)
end

# Plot results (log-log scale)
plot(problem_sizes, cpu_times, label="CPU", linewidth=2, marker=:circle, xscale=:log10, yscale=:log10)
plot!(problem_sizes, gpu_times, label="GPU", linewidth=2, marker=:square)
xlabel!("Problem Size (num_qbits)")
ylabel!("Execution Time (s)")
title!("CPU vs GPU Performance")
savefig("benchmark_plot.png")
=#

In [ ]:
# Ensure CPU is single-threaded
#it should be noted we select 2 threads here because we are running on the Unity cluster. 
#It is advised, prior to startup, that at least 2 CPU threads are used so one can focus on simply running the webapplication. 
#Thus we select 2 threads, also when I tried to run with only 1 it would fail, so that extra thread seems to be focused so
BLAS.set_num_threads(2)

# Define problem sizes (make sure these match the sizes we actually use)
problem_sizes = [64, 512, 1024, 2000, 5000,10000, 20000]
cpu_times = Float64[]
gpu_times = Float64[]

In [ ]:
#64 qubit benchmark
num_qbits=64
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = random_clifford(num_qbits)
s = random_stabilizer(num_qbits)

c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
#512 qubit benchmark
num_qbits=512
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = random_clifford(num_qbits)
s = random_stabilizer(num_qbits)

c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
#1024 qubit benchmark
num_qbits=1024
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = random_clifford(num_qbits)
s = random_stabilizer(num_qbits)

c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
#2000 qubit benchmark
num_qbits=2000
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = tensor_pow(random_clifford(1000),num_qbits/1000)
s = tensor_pow(random_stabilizer(1000),num_qbits/1000)
c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
#5000 qubit benchmark
num_qbits=5000
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = tensor_pow(random_clifford(1000),num_qbits/1000)
s = tensor_pow(random_stabilizer(1000),num_qbits/1000)
c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
#10000 qubit benchmark
num_qbits=10000
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = tensor_pow(random_clifford(1000),num_qbits/1000)
s = tensor_pow(random_stabilizer(1000),num_qbits/1000)
c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
#20000 qubit benchmark
num_qbits=20000
println("\nBenchmarking with num_qbits = $num_qbits")

cliff = tensor_pow(random_clifford(1000),num_qbits/1000)
s = tensor_pow(random_stabilizer(1000),num_qbits/1000)
c_gpu = CuArray(cliff.tab.xzs)  
s_gpu = CuArray(s.tab.xzs)

# GPU Benchmark
gpu_bench = @benchmark CUDA.@sync apply_gate_gpu_64plus!(s_gpu, c_gpu, num_qbits)
gpu_time = median(gpu_bench.times) / 1e9  # Convert to seconds
println("GPU time: $gpu_time seconds")
push!(gpu_times, gpu_time)
show(stdout, "text/plain", gpu_bench)
println()
# CPU Benchmark
cpu_bench = @benchmark apply!(s, cliff, phases=false)
cpu_time = median(cpu_bench.times) / 1e9  # Convert to seconds
println("CPU time: $cpu_time seconds")
push!(cpu_times, cpu_time)
show(stdout, "text/plain", cpu_bench)
println()

In [ ]:
using Plots
plot(problem_sizes, cpu_times, label="CPU", linewidth=2, marker=:circle, xscale=:log10, yscale=:log10)
plot!(problem_sizes, gpu_times, label="GPU", linewidth=2, marker=:square)
xlabel!("Problem Size (num_qbits)")
ylabel!("Execution Time (s)")
title!("CPU vs GPU Performance")
savefig("benchmark_plot.png")

In [ ]:
problem_sizes = [64, 512, 1024, 2000, 5000,20000]
# Plot results (log-log scale)
plot(problem_sizes, cpu_times, label="CPU", linewidth=2, marker=:circle, xscale=:log10, yscale=:log10)
plot!(problem_sizes, gpu_times, label="GPU", linewidth=2, marker=:square)
xlabel!("Problem Size (num_qbits)")
ylabel!("Execution Time (s)")
title!("CPU vs GPU Performance")
savefig("benchmark_plot.png")

In [ ]:
#now requires a pass in of t as the number of threads (per block) and b as number of blocks (per grid)
function apply_gate_gpu_64plus_threadchoice!(s, c, num_bits, t, b)
    #z_start is where the z's start for example if there are 67 bits it will be 3, if there are 129 bits it will be 4 etc.
    #x_stop is just 1 before z_start
    function kernel(s, c, num_bits, numBitstrings, tempXZArr, x_stop, z_start)
        i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
        if i <= num_bits
            for j in 1:64
                #x's loop
                for k in 1:x_stop
                    if s[k, i] & 1 == 1
                        for l in 1:numBitstrings
                            #before it was just c[l, j], but now a bit more complicated indexing
                            tempXZArr[i, l] ⊻= c[l, j+(k-1)*64]
                        end
                    end
                    s[k, i] >>= 1
                end
                #z's loop
                for k in z_start:numBitstrings
                    if s[k, i] & 1 == 1
                        for l in 1:numBitstrings
                             #before it was just c[l, j+ num_bits], but now a bit more complicated
                            tempXZArr[i, l] ⊻= c[l, j+num_bits+(k-z_start)*64]
                        end
                    end
                    s[k, i] >>= 1
                end
            end
            for a in 1:numBitstrings
                s[a, i] = tempXZArr[i, a]
            end
        end
        return 
    end


    # Launch the kernel
    #currently we are passed in number of bits, which I don't think is unreasonable, although could be done away with, with some calculations)
    numBitstrings = ceil(Int, num_bits/64)*2
    x_stop = Int(numBitstrings / 2)
    z_start = Int((numBitstrings / 2) + 1)
    tempXZArr = CUDA.fill(UInt64(0), num_bits, numBitstrings)
    #starting XZArr at 0's
    @cuda threads=t blocks=b kernel(s, c, num_bits, numBitstrings, tempXZArr, x_stop, z_start)
end

In [ ]:
# Benchmarking apply_gate_gpu with varying GPU parameters
num_qbits = 512
cliff = random_clifford(num_qbits)
c_gpu = CuArray(cliff.tab.xzs)  # Copy `c` to GPU memory once
s = random_stabilizer(num_qbits)
s_gpu = CuArray(s.tab.xzs)

println("Benchmarking GPU: Threads per Block = 32, Blocks per Grid = 4")
# Define your kernel launch configuration
gpu_time = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 4)
println("GPU Time: $gpu_time")
show(stdout, "text/plain", gpu_time)



In [ ]:
using CUDA
using Plots

# Function to benchmark for a specific configuration
function benchmark_gpu(num_qbits)
    cliff = random_clifford(num_qbits)
    c_gpu = CuArray(cliff.tab.xzs) 
    s = random_stabilizer(num_qbits)
    s_gpu = CuArray(s.tab.xzs)

    # Explicit configurations for threads per block and blocks per grid
    println("Benchmarking GPU: Threads per Block = 32, Blocks per Grid = 1")
    gpu_time_32_1 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 1)
    println("GPU Times: $gpu_time_32_1")
    median_time_32_1 = median(gpu_time_32_1)
    println("Median Time: $median_time_32_1")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 32, Blocks per Grid = 2")
    gpu_time_32_2 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 2)
    println("GPU Times: $gpu_time_32_2")
    median_time_32_2 = median(gpu_time_32_2)
    println("Median Time: $median_time_32_2")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 32, Blocks per Grid = 4")
    gpu_time_32_4 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 4)
    println("GPU Times: $gpu_time_32_4")
    median_time_32_4 = median(gpu_time_32_4)
    println("Median Time: $median_time_32_4")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 32, Blocks per Grid = 8")
    gpu_time_32_8 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 8)
    println("GPU Times: $gpu_time_32_8")
    median_time_32_8 = median(gpu_time_32_8)
    println("Median Time: $median_time_32_8")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 64, Blocks per Grid = 1")
    gpu_time_64_1 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 1)
    println("GPU Times: $gpu_time_64_1")
    median_time_64_1 = median(gpu_time_64_1)
    println("Median Time: $median_time_64_1")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 64, Blocks per Grid = 2")
    gpu_time_64_2 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 2)
    println("GPU Times: $gpu_time_64_2")
    median_time_64_2 = median(gpu_time_64_2)
    println("Median Time: $median_time_64_2")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 64, Blocks per Grid = 4")
    gpu_time_64_4 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 4)
    println("GPU Times: $gpu_time_64_4")
    median_time_64_4 = median(gpu_time_64_4)
    println("Median Time: $median_time_64_4")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 64, Blocks per Grid = 8")
    gpu_time_64_8 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 8)
    println("GPU Times: $gpu_time_64_8")
    median_time_64_8 = median(gpu_time_64_8)
    println("Median Time: $median_time_64_8")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 128, Blocks per Grid = 1")
    gpu_time_128_1 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 1)
    println("GPU Times: $gpu_time_128_1")
    median_time_128_1 = median(gpu_time_128_1)
    println("Median Time: $median_time_128_1")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 128, Blocks per Grid = 2")
    gpu_time_128_2 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 2)
    println("GPU Times: $gpu_time_128_2")
    median_time_128_2 = median(gpu_time_128_2)
    println("Median Time: $median_time_128_2")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 128, Blocks per Grid = 4")
    gpu_time_128_4 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 4)
    println("GPU Times: $gpu_time_128_4")
    median_time_128_4 = median(gpu_time_128_4)
    println("Median Time: $median_time_128_4")
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    println("Benchmarking GPU: Threads per Block = 128, Blocks per Grid = 8")
    gpu_time_128_8 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 8)
    println("GPU Times: $gpu_time_128_8")
    median_time_128_8 = median(gpu_time_128_8)
    println("Median Time: $median_time_128_8")

    # Store all the results in a dictionary
    results = Dict(
        (32, 1) => (gpu_time_32_1, median_time_32_1), (32, 2) => (gpu_time_32_2, median_time_32_2), 
        (32, 4) => (gpu_time_32_4, median_time_32_4), (32, 8) => (gpu_time_32_8, median_time_32_8),
        (64, 1) => (gpu_time_64_1, median_time_64_1), (64, 2) => (gpu_time_64_2, median_time_64_2), 
        (64, 4) => (gpu_time_64_4, median_time_64_4), (64, 8) => (gpu_time_64_8, median_time_64_8),
        (128, 1) => (gpu_time_128_1, median_time_128_1), (128, 2) => (gpu_time_128_2, median_time_128_2), 
        (128, 4) => (gpu_time_128_4, median_time_128_4), (128, 8) => (gpu_time_128_8, median_time_128_8)
    )

    return results
end

# Run the benchmark for a chosen number of qubits (e.g., 512)
results = benchmark_gpu(512)


In [ ]:
using Plots
using BenchmarkTools
using CUDA
#set same num_qbits at the top as well, so doesn't get over-ridden by potential global variable
num_qbits=512
# Function to benchmark for a specific configuration
function benchmark_gpu(num_qbits)
    cliff = random_clifford(num_qbits)
    c_gpu = CuArray(cliff.tab.xzs) 
    s = random_stabilizer(num_qbits)
    s_gpu = CuArray(s.tab.xzs)
    # Define an array to store the median times
    median_times = Float64[]

    # Benchmarking for Threads per Block = 32
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_32_1 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 1)
    push!(median_times, median(gpu_time_32_1).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_32_2 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 2)
    push!(median_times, median(gpu_time_32_2).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_32_4 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 4)
    push!(median_times, median(gpu_time_32_4).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_32_8 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 32, 8)
    push!(median_times, median(gpu_time_32_8).time / 1e6)  # Convert to seconds

    # Benchmarking for Threads per Block = 64
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_64_1 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 1)
    push!(median_times, median(gpu_time_64_1).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_64_2 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 2)
    push!(median_times, median(gpu_time_64_2).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_64_4 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 4)
    push!(median_times, median(gpu_time_64_4).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_64_8 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 64, 8)
    push!(median_times, median(gpu_time_64_8).time / 1e6)  # Convert to seconds

    # Benchmarking for Threads per Block = 128
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_128_1 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 1)
    push!(median_times, median(gpu_time_128_1).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_128_2 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 2)
    push!(median_times, median(gpu_time_128_2).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_128_4 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 4)
    push!(median_times, median(gpu_time_128_4).time / 1e6)  # Convert to seconds
    
    c_gpu = CuArray(cliff.tab.xzs)
    s_gpu = CuArray(s.tab.xzs)
    gpu_time_128_8 = @benchmark CUDA.@sync apply_gate_gpu_64plus_threadchoice!(s_gpu, c_gpu, num_qbits, 128, 8)
    push!(median_times, median(gpu_time_128_8).time / 1e6)  # Convert to seconds

    return median_times
end

# Run the benchmark for a chosen number of qubits (e.g., 512)
median_times = benchmark_gpu(512)

# Prepare the data for plotting
threads_per_block_values = [32, 64, 128]
blocks_per_grid_values = [1, 2, 4, 8]

# Reshape the data for plotting (grid rows for blocks_per_grid and columns for threads_per_block)
plot_data = reshape(median_times, length(blocks_per_grid_values), length(threads_per_block_values))

# Plotting the results
p = heatmap(blocks_per_grid_values, threads_per_block_values, plot_data,
            xlabel="Blocks per Grid", ylabel="Threads per Block",
            title="GPU Time vs Threads/Blocks Configuration", color=:viridis)

# Show the plot
display(p)
